In [1]:
import os
import sys

# Add project root to PYTHONPATH automatically
PROJECT_ROOT = r"C:\Users\adib4\OneDrive\Documents\Projets perso\CongestionAI\find_issues.ipynb"
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

from src.model_pipelines.dl_pipeline import train_model, evaluate, predict
from src.utils.model_evaluation import evaluate_and_plot_block
from src.utils.hist_baseline import historical_baseline_multi
from src.utils.preprocessing import cyclical_encode, scale_features, encode_detectors
from src.utils.sequences import create_nhits_sequences, NHitsDataset
from src.utils.plots import plot_training_curves
from src.model_pipelines.losses import (
    SpikeWeightedMSELoss,
    TwoTermSpikeLoss,
    DeltaLoss,
    LossConfig,
    create_loss
)

from src.utils.crafted_features import (
    SpikeFeatureConfig,
    add_spike_features,
    add_lags_and_drop
)



from src.models.n_hits import NHitsForecaster
from src.models.tcn_forecaster import MultiHeadTCNForecaster

FILE_PATH = "prepared_data/preprocessed_full_data.csv"

In [2]:
def run_dl_experiment(
    model,
    optimizer,
    criterion,
    X_train_hist,
    Y_train,
    train_det_idx,
    X_val_hist,
    Y_val,
    val_det_idx,
    X_test_hist,
    Y_test,
    test_det_idx,
    device="cuda",
    batch_size=128,
    epochs=10,
    grad_clip=1.0,
    scheduler=None,
    scaler=None,
    exp_name="",
    patience=None,
):
    """
    Runs the full deep-learning training pipeline.

    - Builds model + dataloaders
    - Trains the model
    - Returns model, predictions, losses
    """

    # -------------------------
    # DATALOADERS
    # -------------------------
    train_loader = DataLoader(
        NHitsDataset(X_train_hist, Y_train, train_det_idx),
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        pin_memory=True
    )

    val_loader = DataLoader(
        NHitsDataset(X_val_hist, Y_val, val_det_idx),
        batch_size=batch_size,
        shuffle=False,
        pin_memory=True
    )

    if X_test_hist is None or Y_test is None or test_det_idx is None:
        test_loader = None
    else:
        test_loader = DataLoader(
            NHitsDataset(X_test_hist, Y_test, test_det_idx),
            batch_size=batch_size,
            shuffle=False,
            pin_memory=True
        )

    model.to(device)

    # -------------------------
    # TRAINING
    # -------------------------
    train_losses, val_losses, best_state = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        scaler=scaler,
        device=device,
        num_epochs=epochs,
        grad_clip=grad_clip,
        patience=patience,
    )

    # Save losses to file
    os.makedirs(f"plots_training_dl/{broad_exp_name}/", exist_ok=True)
    with open(f"plots_training_dl/{broad_exp_name}/losses_{exp_name}.txt", "w") as f:
        f.write("epoch,train_loss,val_loss\n")
        for i, (t_loss, v_loss) in enumerate(zip(train_losses, val_losses)):
            f.write(f"{i+1},{t_loss:.6f},{v_loss:.6f}\n")

    plot_training_curves(train_losses, val_losses, filename=f"training_curve{exp_name}.png", dir = f"plots_training_dl/{broad_exp_name}/")
    #model.load_state_dict(best_state)
    if test_loader is not None:
        _, test_loss = evaluate(model, test_loader, criterion, device)
        print(f"Test Loss ({exp_name}): {test_loss:.4f}")

    return model, train_losses, val_losses

In [3]:
def prepare_eval_df(df, idx_seq, preds, horizon):
    """
    df: the dataset (train, val or test)
    idx_seq: array of starting indices returned by create_nhits_sequences
    preds: model predictions (N, horizon)
    """

    df_subset = df.loc[idx_seq].copy()
    print(df.info())
    print(df_subset.info())
    print(len(df_subset), len(idx_seq), preds.shape)

    eval_df = pd.DataFrame({
        "row_idx": idx_seq,
        "timestamp": df_subset["timestamp"].values,
        "detector_id": df_subset["detector_id"].values,
    })
    

    # Add predictions
    for h in range(1, horizon + 1):
        eval_df[f"pred_{h}h"] = preds[:, h-1].numpy()

    # Add ground truth targets
    for h in range(1, horizon + 1):
        eval_df[f"future_{h}h"] = (
            df.groupby("detector_id")["congestion_index"]
              .shift(-h)
              .loc[idx_seq]
              .values
        )

    return eval_df.dropna()

In [4]:
def prepare_dl_data_with_spikes(history_offsets, forecast_horizon, nb_detectors, df_base,
                                years_split, feature_cols_norm, feature_cols_base,
                                weather_lags, spike_config=None):
    """Extended data prep with optional spike features."""
    
    print("Loading data...")
    df_small = df_base[df_base["detector_id"].isin(df_base["detector_id"].unique()[:nb_detectors])].copy()
    df_small = df_small.sort_values(["detector_id", "timestamp"])
    
    # Season encoding
    df_small.loc[(df_small["month"] <= 2) | (df_small["month"] == 12), "season"] = 0
    df_small.loc[(df_small["month"] > 2) & (df_small["month"] <= 5), "season"] = 1
    df_small.loc[(df_small["month"] > 5) & (df_small["month"] <= 8), "season"] = 2
    df_small.loc[(df_small["month"] > 8) & (df_small["month"] <= 11), "season"] = 3
    
    # Add spike features if configured
    feature_cols = feature_cols_base.copy()
    feature_cols_norm_full = feature_cols_norm.copy()
    
    if spike_config is not None:
        print(f"Adding spike features: deltas={spike_config.enable_deltas}, rolling={spike_config.enable_rolling_stats}")
        df_small = add_spike_features(df_small, spike_config)
        spike_feature_cols = spike_config.get_feature_columns()
        spike_norm_cols = spike_config.get_normalization_columns()
        feature_cols = feature_cols + spike_feature_cols
        feature_cols_norm_full = feature_cols_norm_full + spike_norm_cols
        print(f"  Added columns: {spike_feature_cols}")
    
    # Detector encoding
    df_small, det2idx = encode_detectors(df_small)
    
    # Add weather lag column names
    if "temperature" in feature_cols:
        feature_cols = feature_cols + [f"temperature_lag_{lag}h" for lag in weather_lags] \
            + [f"precipitation_lag_{lag}h" for lag in weather_lags] \
            + [f"visibility_lag_{lag}h" for lag in weather_lags]
    
    # Split
    train = df_small[df_small["timestamp"].dt.year.isin(years_split[0])].copy()
    val = df_small[df_small["timestamp"].dt.year.isin(years_split[1])].copy()
    test = df_small[df_small["timestamp"].dt.year.isin(years_split[2])].copy() if years_split[2] else None
    
    train = train.set_index("orig_idx")
    val = val.set_index("orig_idx")
    if test is not None:
        test = test.set_index("orig_idx")
    
    # Normalization
    minmax_cols = ["lon", "lat", "year", "season"]
    train, val, test, std_scaler, mm_scaler = scale_features(
        train, val, test, feature_cols_norm_full, latlon_cols=minmax_cols
    )
    
    # Weather lags
    if "temperature" in feature_cols_base:
        train = add_lags_and_drop(train, weather_lags)
        val = add_lags_and_drop(val, weather_lags)
        if test is not None:
            test = add_lags_and_drop(test, weather_lags)
    
    # Drop NaNs from spike features
    if spike_config is not None:
        spike_cols_in_df = [c for c in spike_feature_cols if c in train.columns]
        train = train.dropna(subset=spike_cols_in_df)
        val = val.dropna(subset=spike_cols_in_df)
        if test is not None:
            test = test.dropna(subset=spike_cols_in_df)
    
    # Keep only needed columns (congestion_index is already in feature_cols)
    keep_cols = feature_cols + ["timestamp", "detector_id", "det_index"]
    keep_cols = [c for c in keep_cols if c in train.columns]
    
    train = train[keep_cols]
    val = val[keep_cols]
    if test is not None:
        test = test[keep_cols]
    
    # Build sequences
    X_train_hist, Y_train, idx_train, det_train = create_nhits_sequences(
        train, feature_cols, history_offsets, forecast_horizon)
    X_val_hist, Y_val, idx_val, det_val = create_nhits_sequences(
        val, feature_cols, history_offsets, forecast_horizon)
    
    if test is not None:
        X_test_hist, Y_test, idx_test, det_test = create_nhits_sequences(
            test, feature_cols, history_offsets, forecast_horizon)
    else:
        X_test_hist, Y_test, idx_test, det_test = None, None, None, None
    
    print(f"Sequences created. Features: {len(feature_cols)}, Train samples: {len(Y_train)}")
    
    return (X_train_hist, Y_train, idx_train, det_train,
            X_val_hist, Y_val, idx_val, det_val,
            X_test_hist, Y_test, idx_test, det_test,
            train, val, test, std_scaler, mm_scaler)

In [5]:
def main(nb_detectors, forecast_horizon, history_offsets, exp_name, 
                   df_base, feature_cols_norm, feature_cols, weather_lags, model_config=None,
                   years_split=([2019,2020,2021,2022,2023,2024], [2018], [2016]),
                   evaluation_years=None, spike_config=None,
                   spike_eval_threshold=0.38, criterion=None, epochs=10,
                   optim_config=None, schedule_config=None):
    
    dir = f"plots_training_dl/{broad_exp_name}/"

    X_train_hist, Y_train, idx_train, det_train, \
    X_val_hist, Y_val, idx_val, det_val, \
    X_test_hist, Y_test, idx_test, det_test, \
    train, val, test, \
    std_scaler, mm_scaler = prepare_dl_data_with_spikes(history_offsets, 
                                            forecast_horizon, 
                                            nb_detectors, df_base,
                                            feature_cols_norm=feature_cols_norm, 
                                            feature_cols_base=feature_cols, 
                                            weather_lags=weather_lags,
                                            years_split=years_split,
                                            spike_config=spike_config)
    
    if criterion is None:
        criterion = nn.MSELoss()
    model = MultiHeadTCNForecaster(**model_config, num_features=X_train_hist.shape[-1])    
    batch_size = 512
    if optim_config is None:
        optim = torch.optim.Adam(model.parameters(), lr=lr)
        lr = 1e-4
    else:
        if optim_config["type"].lower() == "adam":
            optim = torch.optim.Adam(model.parameters(), lr=optim_config.get("lr", 0.0001))
        elif optim_config["type"].lower() == "adamw":
            optim = torch.optim.AdamW(model.parameters(), lr=optim_config.get("lr", 0.0001), 
                    weight_decay=optim_config.get("weight_decay", 0), betas=optim_config.get("betas", (0.9, 0.999)))
        else:
            raise ValueError(f"Unsupported optimizer type: {optim_config['type']}")
    if schedule_config is not None:
        if schedule_config.get("type") == "OneCycleLR":
            steps_per_epoch = len(Y_train) // batch_size
            schedule_config["steps_per_epoch"] = steps_per_epoch
        sched_class = getattr(torch.optim.lr_scheduler, schedule_config["type"])
        sched_params = {k: v for k, v in schedule_config.items() if k != "type"}
        scheduler = sched_class(optim, **sched_params)
    else:
        scheduler = None

    params_experiment = {
        "model": model,
        "optimizer": optim,
        "criterion": criterion,
        "X_train_hist": X_train_hist,
        "Y_train": Y_train,
        "train_det_idx": det_train,
        "X_val_hist": X_val_hist,
        "Y_val": Y_val,
        "val_det_idx": det_val,
        "X_test_hist": X_test_hist,
        "Y_test": Y_test,
        "test_det_idx": det_test,
        "device": "cuda",
        "batch_size": batch_size,
        "epochs": epochs,
        "grad_clip": None,
        "scheduler": scheduler
    }
        
    if evaluation_years is None:
        evaluation_years = years_split[1]
    
    # RUN EXPERIMENT
    model, train_losses, val_losses = run_dl_experiment(**params_experiment, exp_name=exp_name)
    eval_df = prepare_eval_df(val, idx_val, predict(model, X_val_hist, det_val), forecast_horizon)
    eval_df["congestion_index"] = val.loc[idx_val, "congestion_index"].values
    metrics = evaluate_and_plot_block(eval_df, horizon=forecast_horizon, years=evaluation_years, plot_years=evaluation_years, 
                            filename=exp_name,
                            dir=dir, max_blocks=15,
                            eval_spikes=True,
                            spike_threshold=spike_eval_threshold)

    # BUILD RESULT DICT - only computed values, not input params
    result = {
        "exp_name": exp_name,
        "num_features": X_train_hist.shape[-1],
        "train_samples": len(Y_train),
        "val_samples": len(Y_val),
        "final_train_loss": train_losses[-1],
        "final_val_loss": val_losses[-1],
        "best_val_loss": min(val_losses),
        "best_epoch": val_losses.index(min(val_losses)) + 1,
    }
    
    # Add all metrics from evaluate_and_plot_block
    if metrics is not None:
        for k, v in metrics.items():
            if isinstance(v, (int, float)):
                result[k] = v
    
    return result, model, (train_losses, val_losses)



In [6]:
df_base = pd.read_csv(FILE_PATH)
df_base["timestamp"] = pd.to_datetime(df_base["timestamp"])
df_base["orig_idx"] = df_base.index
df_base = cyclical_encode(df_base)

In [24]:
# Fixed parameters
feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed"
]
nb_detectors = 20
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]

# Fixed for now
forecast_horizon = 24
h_offsets = list(range(24))
w_lags = [-1, -2, -6, -8]
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
spike_config = SpikeFeatureConfig(
        enable_deltas=True,
        enable_abs_deltas=False,
        enable_rolling_stats=False,
        delta_lags=[1, 2, 4, 6],
)
cfg_loss = LossConfig(loss_type="spike_weighted", spike_weight=3.0, spike_threshold=spike_trigger_threshold)
broad_exp_name = "TCN-spike-scaled_24horizon"

criterion = create_loss(cfg_loss)
exp_name = "base_parameters-nothing"
model_config = {"horizon": forecast_horizon,
                "num_detectors": nb_detectors,
                "emb_dim": 256,
                "num_channels": (128, 256, 256),
                "kernel_size": 3,
                "dropout": 0.1,
                "use_se": False,
                "pooling": "last"
            }

main(nb_detectors=nb_detectors,
     forecast_horizon=forecast_horizon,
     history_offsets=h_offsets,
     exp_name=exp_name,
     df_base=df_base,
     feature_cols_norm=feature_cols_norm_base,
     feature_cols=feature_cols_base,
     weather_lags=w_lags,
     model_config=model_config,
     years_split=years_split,
     evaluation_years=[2019],
     spike_config=spike_config,
     spike_eval_threshold=eval_spike_threshold,
     criterion=criterion)


Loading data...
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h']
Sequences created. Features: 31, Train samples: 1068231


Validating: 100%|██████████| 316/316 [00:03<00:00, 87.63it/s]


Epoch 1/15 - Train Loss: 0.1989 - Val Loss: 0.1342


Validating: 100%|██████████| 316/316 [00:03<00:00, 88.68it/s]


Epoch 2/15 - Train Loss: 0.1557 - Val Loss: 0.1248


...

Epoch 13/15 - Train Loss: 0.1198 - Val Loss: 0.1211


Validating: 100%|██████████| 316/316 [00:03<00:00, 85.72it/s]


Epoch 14/15 - Train Loss: 0.1179 - Val Loss: 0.1206


Validating: 100%|██████████| 316/316 [00:03<00:00, 83.41it/s]


Epoch 15/15 - Train Loss: 0.1162 - Val Loss: 0.1217
<class 'pandas.core.frame.DataFrame'>
Index: 162300 entries, 10086243 to 13113463
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   hour_sin               162300 non-null  float64       
 1   hour_cos               162300 non-null  float64       
 2   dow_sin                162300 non-null  float64       
 3   dow_cos                162300 non-null  float64       
 4   month_sin              162300 non-null  float64       
 5   month_cos              162300 non-null  float64       
 6   lon                    162300 non-null  float64       
 7   lat                    162300 non-null  float64       
 8   year                   162300 non-null  float64       
 9   season                 162300 non-null  float64       
 10  temperature            162300 non-null  float64       
 11  precipitation          162300 non-null  float64 

In [26]:
# Experiment model depth / width / kernel / pooling / SE

broad_exp_name = "Experiment_TCN_complexity-24horizon"

feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed"
]
nb_detectors = 20
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]

# Fixed for now
forecast_horizon = 24
h_offsets = list(range(24))
w_lags = [-1, -2, -6, -8]
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
spike_config = SpikeFeatureConfig(
        enable_deltas=True,
        enable_abs_deltas=False,
        enable_rolling_stats=False,
        delta_lags=[1, 2, 4, 6],
)
cfg_loss = LossConfig(loss_type="spike_weighted", spike_weight=3.0, spike_threshold=spike_trigger_threshold)

experiments = [
    # BASELINE REFERENCE
    {"name": "BASELINE", "num_channels": (128, 256, 256), "kernel_size": 3, "pooling": "last", "use_se": False},
    
    # DEPTH / WIDTH SWEEP
    {"name": "DEPTH_4x256", "num_channels": (128, 256, 256, 256), "kernel_size": 3, "pooling": "last", "use_se": False},
    {"name": "DEPTH_3x512", "num_channels": (128, 256, 512), "kernel_size": 3, "pooling": "last", "use_se": False},
    {"name": "DEPTH_4x512", "num_channels": (128, 256, 512, 512), "kernel_size": 3, "pooling": "last", "use_se": False},
    
    # KERNEL SIZE SWEEP
    {"name": "K5", "num_channels": (128, 256, 256), "kernel_size": 5, "pooling": "last", "use_se": False},
    {"name": "K7", "num_channels": (128, 256, 256), "kernel_size": 7, "pooling": "last", "use_se": False},
    {"name": "DEEP_K5", "num_channels": (128, 256, 256, 256), "kernel_size": 5, "pooling": "last", "use_se": False},
    {"name": "DEEP_K7", "num_channels": (128, 256, 256, 256), "kernel_size": 7, "pooling": "last", "use_se": False},
    
    # POOLING EXPERIMENTS
    {"name": "ATTENTION_POOL", "num_channels": (128, 256, 256), "kernel_size": 3, "pooling": "attention", "use_se": False},
    {"name": "ATTENTION_K5", "num_channels": (128, 256, 256), "kernel_size": 5, "pooling": "attention", "use_se": False},
    {"name": "ATTENTION_DEEP", "num_channels": (128, 256, 256, 256), "kernel_size": 3, "pooling": "attention", "use_se": False},
    
    # SE BLOCK VARIANTS
    {"name": "SE_BASE", "num_channels": (128, 256, 256), "kernel_size": 3, "pooling": "last", "use_se": True},
    {"name": "SE_DEEP", "num_channels": (128, 256, 256, 256), "kernel_size": 3, "pooling": "last", "use_se": True},
    {"name": "SE_ATTENTION", "num_channels": (128, 256, 256), "kernel_size": 3, "pooling": "attention", "use_se": True},
]

results = []

for exp in experiments:
    exp_name = exp["name"]
    print(f"\n{'='*60}\nRunning: {exp_name}\n{'='*60}")
    criterion = create_loss(cfg_loss)
    
    model_config = {
        "horizon": forecast_horizon,
        "num_detectors": nb_detectors,
        "emb_dim": 256,
        "num_channels": exp["num_channels"],
        "kernel_size": exp["kernel_size"],
        "dropout": 0.1,
        "use_se": exp["use_se"],
        "pooling": exp["pooling"]
    }
    
    try:
        result, model, losses = main(
            nb_detectors=nb_detectors,
            forecast_horizon=forecast_horizon,
            history_offsets=h_offsets,
            exp_name=exp_name,
            df_base=df_base,
            feature_cols_norm=feature_cols_norm_base,
            feature_cols=feature_cols_base,
            weather_lags=w_lags,
            model_config=model_config,
            years_split=years_split,
            evaluation_years=[2019],
            spike_config=spike_config,
            spike_eval_threshold=eval_spike_threshold,
            criterion=criterion
        )
        
        # Add experiment config to result
        result["num_channels"] = str(exp["num_channels"])
        result["kernel_size"] = exp["kernel_size"]
        result["pooling"] = exp["pooling"]
        result["use_se"] = exp["use_se"]
        results.append(result)
        
        print(f"✓ {exp_name}: val_loss={result['best_val_loss']:.4f}")
        
    except Exception as e:
        print(f"✗ FAILED: {exp_name} - {e}")
        results.append({"exp_name": exp_name, "error": str(e)})
    
    torch.cuda.empty_cache()

# Save results
os.makedirs(f"plots_training_dl/{broad_exp_name}/", exist_ok=True)
results_df = pd.DataFrame(results)
results_df.to_csv(f"plots_training_dl/{broad_exp_name}/grid_results.csv", index=False)

print("\n\n" + "="*60)
print("EXPERIMENTS COMPLETE")
print("="*60)
print(results_df[["exp_name", "num_channels", "kernel_size", "pooling", "use_se", "best_val_loss"]].to_string())


Running: BASELINE
Loading data...
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h']
Sequences created. Features: 31, Train samples: 1068231


Validating: 100%|██████████| 316/316 [00:03<00:00, 88.39it/s]


Epoch 1/25 - Train Loss: 0.1985 - Val Loss: 0.1309


Validating: 100%|██████████| 316/316 [00:03<00:00, 84.13it/s]


Epoch 2/25 - Train Loss: 0.1557 - Val Loss: 0.1242


...

Epoch 23/25 - Train Loss: 0.0933 - Val Loss: 0.1297


Validating: 100%|██████████| 316/316 [00:03<00:00, 81.88it/s]


Epoch 24/25 - Train Loss: 0.0922 - Val Loss: 0.1318


Validating: 100%|██████████| 316/316 [00:03<00:00, 79.19it/s]


Epoch 25/25 - Train Loss: 0.0913 - Val Loss: 0.1319
<class 'pandas.core.frame.DataFrame'>
Index: 162300 entries, 10086243 to 13113463
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   hour_sin               162300 non-null  float64       
 1   hour_cos               162300 non-null  float64       
 2   dow_sin                162300 non-null  float64       
 3   dow_cos                162300 non-null  float64       
 4   month_sin              162300 non-null  float64       
 5   month_cos              162300 non-null  float64       
 6   lon                    162300 non-null  float64       
 7   lat                    162300 non-null  float64       
 8   year                   162300 non-null  float64       
 9   season                 162300 non-null  float64       
 10  temperature            162300 non-null  float64       
 11  precipitation          162300 non-null  float64 

In [ ]:
# Experiment: History length & Weather lags (fixed best architecture from previous exp)

broad_exp_name = "Experiment_TCN_history_weather-24horizon"

feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed"
]
nb_detectors = 20
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]

forecast_horizon = 24
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
spike_config = SpikeFeatureConfig(
        enable_deltas=True,
        enable_abs_deltas=False,
        enable_rolling_stats=False,
        delta_lags=[1, 2, 4, 6],
)
cfg_loss = LossConfig(loss_type="spike_weighted", spike_weight=3.0, spike_threshold=spike_trigger_threshold)

# Fixed architecture from previous experiment (DEEP_K7)
model_config = {
    "horizon": forecast_horizon,
    "num_detectors": nb_detectors,
    "emb_dim": 256,
    "num_channels": (128, 256, 256, 256),
    "kernel_size": 7,
    "dropout_encoder": 0.1,
    "dropout_heads": 0.1,
    "use_se": False,
    "pooling": "last"
}

experiments = [
    # H24_W8: 24h history, weather lags 0-8h (baseline)
    {"name": "H24_W8", "h_offsets": list(range(24)), "w_lags": [0, -1, -2, -3, -4, -5, -6, -7, -8]},
    
    # H48_W8: 48h history, weather lags 0-8h
    {"name": "H48_W8", "h_offsets": list(range(48)), "w_lags": [0, -1, -2, -3, -4, -5, -6, -7, -8]},
    
    # H24_W24S: 24h history, sparse weather lags [0, 3, 6, 12, 24]
    {"name": "H24_W24S", "h_offsets": list(range(24)), "w_lags": [0, -3, -6, -12, -24]},
    
    # H48_W24S: 48h history, sparse weather lags [0, 3, 6, 12, 24]
    {"name": "H48_W24S", "h_offsets": list(range(48)), "w_lags": [0, -3, -6, -12, -24]},

    {"name": "H48_W24dense", "h_offsets": list(range(48)), "w_lags": -1*list(range(0,25,2))},
]

results = []

for exp in experiments:
    exp_name = exp["name"]
    print(f"\n{'='*60}\nRunning: {exp_name}\n{'='*60}")
    print(f"  History: {len(exp['h_offsets'])}h, Weather lags: {exp['w_lags']}")
    
    criterion = create_loss(cfg_loss)
    
    try:
        result, model, losses = main(
            nb_detectors=nb_detectors,
            forecast_horizon=forecast_horizon,
            history_offsets=exp["h_offsets"],
            exp_name=exp_name,
            df_base=df_base,
            feature_cols_norm=feature_cols_norm_base,
            feature_cols=feature_cols_base,
            weather_lags=exp["w_lags"],
            model_config=model_config,
            years_split=years_split,
            evaluation_years=[2019],
            spike_config=spike_config,
            spike_eval_threshold=eval_spike_threshold,
            criterion=criterion
        )
        
        # Add experiment config to result
        result["history_len"] = len(exp["h_offsets"])
        result["weather_lags"] = str(exp["w_lags"])
        result["n_weather_lags"] = len(exp["w_lags"])
        results.append(result)
        
        print(f"✓ {exp_name}: val_loss={result['best_val_loss']:.4f}")
        
    except Exception as e:
        print(f"✗ FAILED: {exp_name} - {e}")
        results.append({"exp_name": exp_name, "error": str(e)})
    
    torch.cuda.empty_cache()

# Save results
os.makedirs(f"plots_training_dl/{broad_exp_name}/", exist_ok=True)
results_df = pd.DataFrame(results)
results_df.to_csv(f"plots_training_dl/{broad_exp_name}/grid_results.csv", index=False)

print("\n\n" + "="*60)
print("EXPERIMENTS COMPLETE")
print("="*60)
print(results_df[["exp_name", "history_len", "n_weather_lags", "best_val_loss", "best_epoch"]].to_string())


Running: H24_W8
  History: 24h, Weather lags: [0, -1, -2, -3, -4, -5, -6, -7, -8]
Loading data...
Adding spike features: deltas=True, rolling=False
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h']
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h']
Sequences created. Features: 46, Train samples: 1068231
Sequences created. Features: 46, Train samples: 1068231


Validating: 100%|██████████| 316/316 [00:08<00:00, 38.42it/s]


Epoch 1/10 - Train Loss: 0.1902 - Val Loss: 0.1290


Validating: 100%|██████████| 316/316 [00:08<00:00, 37.85it/s]


Epoch 2/10 - Train Loss: 0.1515 - Val Loss: 0.1245


...

Epoch 8/10 - Train Loss: 0.1181 - Val Loss: 0.1232


Validating: 100%|██████████| 315/315 [00:14<00:00, 21.58it/s]


Epoch 9/10 - Train Loss: 0.1134 - Val Loss: 0.1245


Validating: 100%|██████████| 315/315 [00:14<00:00, 21.58it/s]



Epoch 10/10 - Train Loss: 0.1090 - Val Loss: 0.1276
<class 'pandas.core.frame.DataFrame'>
Index: 162460 entries, 10086243 to 13116124
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   hour_sin          162460 non-null  float64       
 1   hour_cos          162460 non-null  float64       
 2   dow_sin           162460 non-null  float64       
 3   dow_cos           162460 non-null  float64       
 4   month_sin         162460 non-null  float64       
 5   month_cos         162460 non-null  float64       
 6   lon               162460 non-null  float64       
 7   lat               162460 non-null  float64       
 8   year              162460 non-null  float64       
 9   season            162460 non-null  float64       
 10  temperature       162460 non-null  float64       
 11  precipitation     162460 non-null  float64       
 12  visibility        162460 non-null  float64       
 13  con

In [ ]:
# Experiment: Scale to 200 detectors with best config (H48_W24S)

broad_exp_name = "Experiment_TCN_scale_40det-24horizon-h48_w24s"

feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed"
]
nb_detectors = 40
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]

forecast_horizon = 24
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
spike_config = SpikeFeatureConfig(
        enable_deltas=True,
        enable_abs_deltas=False,
        enable_rolling_stats=False,
        delta_lags=[1, 2, 4, 6],
)
cfg_loss = LossConfig(loss_type="spike_weighted", spike_weight=3.0, spike_threshold=spike_trigger_threshold)

# Fixed architecture (DEEP_K7)
model_config = {
    "horizon": forecast_horizon,
    "num_detectors": nb_detectors,
    "emb_dim": 256,
    "num_channels": (128, 256, 256, 256),
    "kernel_size": 7,
    "dropout_encoder": 0.1,
    "dropout_heads": 0.1,
    "use_se": False,
    "pooling": "last"
}

# Best config from previous experiment
h_offsets = list(range(48))
w_lags = [0, -3, -6, -12, -24]

criterion = create_loss(cfg_loss)
exp_name = "H48_W24S_200det"

result, model, losses = main(
    nb_detectors=nb_detectors,
    forecast_horizon=forecast_horizon,
    history_offsets=h_offsets,
    exp_name=exp_name,
    df_base=df_base,
    feature_cols_norm=feature_cols_norm_base,
    feature_cols=feature_cols_base,
    weather_lags=w_lags,
    model_config=model_config,
    years_split=years_split,
    evaluation_years=[2019],
    spike_config=spike_config,
    spike_eval_threshold=eval_spike_threshold,
    criterion=criterion,
    epochs=10
)

print(f"\n{'='*60}")
print(f"RESULT: {exp_name}")
print(f"{'='*60}")
print(f"Best val loss: {result['best_val_loss']:.4f}")
print(f"Best epoch: {result['best_epoch']}")
print(f"Train samples: {result['train_samples']}")
print(f"Val samples: {result['val_samples']}")

Loading data...
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h']
Sequences created. Features: 34, Train samples: 2067346


Validating: 100%|██████████| 627/627 [00:24<00:00, 25.44it/s]


Epoch 1/10 - Train Loss: 0.1909 - Val Loss: 0.1342


Validating: 100%|██████████| 627/627 [00:24<00:00, 25.22it/s]


Epoch 2/10 - Train Loss: 0.1530 - Val Loss: 0.1272


...

Epoch 9/10 - Train Loss: 0.1036 - Val Loss: 0.1284


Validating: 100%|██████████| 627/627 [00:29<00:00, 20.90it/s]


Epoch 10/10 - Train Loss: 0.1006 - Val Loss: 0.1288
<class 'pandas.core.frame.DataFrame'>
Index: 323545 entries, 10086243 to 13106972
Data columns (total 37 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   hour_sin                323545 non-null  float64       
 1   hour_cos                323545 non-null  float64       
 2   dow_sin                 323545 non-null  float64       
 3   dow_cos                 323545 non-null  float64       
 4   month_sin               323545 non-null  float64       
 5   month_cos               323545 non-null  float64       
 6   lon                     323545 non-null  float64       
 7   lat                     323545 non-null  float64       
 8   year                    323545 non-null  float64       
 9   season                  323545 non-null  float64       
 10  temperature             323545 non-null  float64       
 11  precipitation           323545 non-

c:\Users\adib4\OneDrive\Documents\Projets perso\CongestionAI\src\utils\model_evaluation.py:32: RuntimeWarning: invalid value encountered in divide
  corr_per_block = np.where(denom == 0, np.nan, num / denom)



SPIKE PERFORMANCE REPORT
Metric                                        Value
-------------------------------------------------------
True spikes                                 704,841
Predicted spikes                            157,415
Spike frequency                              9.16%
-------------------------------------------------------
Spike Recall                                10.16%
Spike Precision                             45.48%
Spike F1                                    16.61%
-------------------------------------------------------
Spike MAE                                    0.4439
Non-Spike MAE                                0.1365
Spike Delta MAE                              0.5649
-------------------------------------------------------
Spike Delta Corr                             0.3853
Overall Delta Corr                           0.2760
Direction Accuracy                          58.11%
Spike Direction Accuracy                    68.91%
320705 total blocks in dataf

In [8]:
# Experiment: Retry engineered binary features

broad_exp_name = "Experiment_TCN-binary_features-20det_24horizon-h24_w24s"

feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed",
    "is_weekend", "is_holiday", "is_school_holiday", "is_rush_hour", "is_snow", "is_fog" # binary features
]
nb_detectors = 20
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]

forecast_horizon = 24
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
spike_config = SpikeFeatureConfig(
        enable_deltas=True,
        enable_abs_deltas=False,
        enable_rolling_stats=False,
        delta_lags=[1, 2, 4, 6],
)
cfg_loss = LossConfig(loss_type="spike_weighted", spike_weight=3.0, spike_threshold=spike_trigger_threshold)

# Fixed architecture (DEEP_K7)
model_config = {
    "horizon": forecast_horizon,
    "num_detectors": nb_detectors,
    "emb_dim": 256,
    "num_channels": (128, 256, 256),
    "kernel_size": 5,
    "dropout_encoder": 0.1,
    "dropout_heads": 0.1,
    "use_se": False,
    "pooling": "last"
}

# Light one for quick test
h_offsets = list(range(24))
w_lags = [0, -3, -6, -12, -24]

criterion = create_loss(cfg_loss)
exp_name = "H24_W24S_20det"

result, model, losses = main(
    nb_detectors=nb_detectors,
    forecast_horizon=forecast_horizon,
    history_offsets=h_offsets,
    exp_name=exp_name,
    df_base=df_base,
    feature_cols_norm=feature_cols_norm_base,
    feature_cols=feature_cols_base,
    weather_lags=w_lags,
    model_config=model_config,
    years_split=years_split,
    evaluation_years=[2019],
    spike_config=spike_config,
    spike_eval_threshold=eval_spike_threshold,
    criterion=criterion,
    epochs=10
)

print(f"\n{'='*60}")
print(f"RESULT: {exp_name}")
print(f"{'='*60}")
print(f"Best val loss: {result['best_val_loss']:.4f}")
print(f"Best epoch: {result['best_epoch']}")
print(f"Train samples: {result['train_samples']}")
print(f"Val samples: {result['val_samples']}")

Loading data...
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h']
Sequences created. Features: 40, Train samples: 1067911


Epoch 1/10 - training:  50%|█████     | 1051/2085 [00:38<00:37, 27.51it/s]


KeyboardInterrupt: 

In [ ]:
# Test more new spike feature

broad_exp_name = "Experiment_TCN-binary_features-20det_24horizon-h24_w24s"

feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed",
    "is_weekend", "is_holiday", "is_school_holiday", "is_rush_hour", "is_snow", "is_fog" # binary features
]
nb_detectors = 20
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]

forecast_horizon = 24
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
spike_config = SpikeFeatureConfig(
        enable_deltas=True,
        enable_abs_deltas=False,
        enable_rolling_stats=False,
        delta_lags=[1, 2, 4, 6],
        enable_volatility=True,
        volatility_window=3,
        volatility_binary_threshold=0.05
)
cfg_loss = LossConfig(loss_type="spike_weighted", spike_weight=3.0, spike_threshold=spike_trigger_threshold)

# Fixed architecture (DEEP_K7)
model_config = {
    "horizon": forecast_horizon,
    "num_detectors": nb_detectors,
    "emb_dim": 256,
    "num_channels": (128, 256, 256),
    "kernel_size": 5,
    "dropout_encoder": 0.1,
    "dropout_heads": 0.1,
    "use_se": False,
    "pooling": "last"
}

# Light one for quick test
h_offsets = list(range(24))
w_lags = [0, -3, -6, -12, -24]

criterion = create_loss(cfg_loss)
exp_name = "rolling_vol_features_added_H24_W24S_20det"

result, model, losses = main(
    nb_detectors=nb_detectors,
    forecast_horizon=forecast_horizon,
    history_offsets=h_offsets,
    exp_name=exp_name,
    df_base=df_base,
    feature_cols_norm=feature_cols_norm_base,
    feature_cols=feature_cols_base,
    weather_lags=w_lags,
    model_config=model_config,
    years_split=years_split,
    evaluation_years=[2019],
    spike_config=spike_config,
    spike_eval_threshold=eval_spike_threshold,
    criterion=criterion,
    epochs=10
)

print(f"\n{'='*60}")
print(f"RESULT: {exp_name}")
print(f"{'='*60}")
print(f"Best val loss: {result['best_val_loss']:.4f}")
print(f"Best epoch: {result['best_epoch']}")
print(f"Train samples: {result['train_samples']}")
print(f"Val samples: {result['val_samples']}")

In [11]:
# Batch experiment: Spike config, loss tuning, architecture, and history variations

import time
import copy



broad_exp_name = "Experiment_TCN_batch_15_spike_loss_arch"

# ========================================================
# FIXED PARAMETERS
# ========================================================
feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed",
    "is_weekend", "is_holiday", "is_school_holiday", "is_rush_hour", "is_snow", "is_fog"
]
nb_detectors = 20
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]
forecast_horizon = 24
eval_spike_threshold = 0.38
epochs = 10

# Defaults
history_offsets_default = list(range(24))
weather_lags_default = [0, -3, -6, -12, -24]

BASE_MODEL_CONFIG = {
    "horizon": 24,
    "num_detectors": 20,
    "emb_dim": 256,
    "num_channels": (128, 256, 256),
    "kernel_size": 5,
    "dropout_encoder": 0.1,
    "dropout_heads": 0.1,
    "use_se": False,
    "pooling": "last"
}

# ========================================================
# EXPERIMENT DEFINITIONS
# ========================================================
experiments = [
    {
        "name": "EXP_spikeThr_0.04__lags_1-2-4-6__noAbs",
        "spike_trigger_threshold": 0.04,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_spikeThr_0.04__lags_1-2-3-4-6-8",
        "spike_trigger_threshold": 0.04,
        "delta_lags": [1, 2, 3, 4, 6, 8],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_spikeThr_0.04__lags_1-3-6",
        "spike_trigger_threshold": 0.04,
        "delta_lags": [1, 3, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_spikeThr_0.03__lags_1-2-4-6",
        "spike_trigger_threshold": 0.03,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_spikeThr_0.06__lags_1-2-4-6",
        "spike_trigger_threshold": 0.06,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_loss_spikeWeight_2",
        "spike_trigger_threshold": 0.15,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 2.0,
        "spike_lambda": None,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_loss_spikeWeight_4",
        "spike_trigger_threshold": 0.15,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 4.0,
        "spike_lambda": None,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_loss_twoTerm_lambda2",
        "spike_trigger_threshold": 0.15,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "two_term",
        "spike_weight": None,
        "spike_lambda": 2.0,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_model_4layers_k7",
        "spike_trigger_threshold": 0.15,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": {**BASE_MODEL_CONFIG, "num_channels": (128, 256, 256, 256), "kernel_size": 7},
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_model_drop02",
        "spike_trigger_threshold": 0.15,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": {**BASE_MODEL_CONFIG, "dropout_encoder": 0.2, "dropout_heads": 0.2},
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_model_poolAvg",
        "spike_trigger_threshold": 0.15,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": {**BASE_MODEL_CONFIG, "pooling": "avg"},
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_hist36",
        "spike_trigger_threshold": 0.15,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": list(range(36)),
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_hist36_denseWeather",
        "spike_trigger_threshold": 0.15,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": list(range(36)),
        "weather_lags": [0, -1, -3, -6, -12, -24],
    },
    {
        "name": "EXP_model_k7_hist36",
        "spike_trigger_threshold": 0.15,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 3.0,
        "spike_lambda": None,
        "model_config": {**BASE_MODEL_CONFIG, "kernel_size": 7},
        "history_offsets": list(range(36)),
        "weather_lags": weather_lags_default,
    },
    {
        "name": "EXP_combined_bestGuess",
        "spike_trigger_threshold": 0.04,
        "delta_lags": [1, 2, 4, 6],
        "enable_abs_deltas": False,
        "loss_type": "spike_weighted",
        "spike_weight": 4.0,
        "spike_lambda": None,
        "model_config": copy.deepcopy(BASE_MODEL_CONFIG),
        "history_offsets": history_offsets_default,
        "weather_lags": weather_lags_default,
    },
]

# ========================================================
# RUN EXPERIMENTS
# ========================================================
results = []

for exp in experiments:
    exp_name = exp["name"]
    print(f"\n{'='*70}")
    print(f"Running: {exp_name}")
    print(f"{'='*70}")
    
    start_time = time.time()
    
    # Build spike config
    spike_config = SpikeFeatureConfig(
        enable_deltas=True,
        enable_abs_deltas=exp["enable_abs_deltas"],
        enable_rolling_stats=False,
        delta_lags=exp["delta_lags"],
        enable_volatility=True,
        volatility_window=3,
        volatility_binary_threshold=0.05
    )
    
    # Build loss using existing LossConfig
    if exp["loss_type"] == "spike_weighted":
        cfg_loss = LossConfig(
            loss_type="spike_weighted",
            spike_weight=exp["spike_weight"],
            spike_threshold=exp["spike_trigger_threshold"]
        )
    elif exp["loss_type"] == "two_term":
        cfg_loss = LossConfig(
            loss_type="two_term",
            spike_lambda=exp["spike_lambda"],
            spike_threshold=exp["spike_trigger_threshold"]
        )
    else:
        cfg_loss = LossConfig(loss_type="mse")
    
    criterion = create_loss(cfg_loss)
    model_config = exp["model_config"].copy()
    
    print(f"  spike_trigger_threshold: {exp['spike_trigger_threshold']}")
    print(f"  delta_lags: {exp['delta_lags']}")
    print(f"  loss_type: {exp['loss_type']}")
    print(f"  spike_weight: {exp['spike_weight']}, spike_lambda: {exp['spike_lambda']}")
    print(f"  history_offsets: {len(exp['history_offsets'])}h")
    print(f"  weather_lags: {exp['weather_lags']}")
    print(f"  model_config: channels={model_config['num_channels']}, k={model_config['kernel_size']}, pool={model_config['pooling']}")
    
    try:
        result, model, losses = main(
            nb_detectors=nb_detectors,
            forecast_horizon=forecast_horizon,
            history_offsets=exp["history_offsets"],
            exp_name=exp_name,
            df_base=df_base,
            feature_cols_norm=feature_cols_norm_base,
            feature_cols=feature_cols_base,
            weather_lags=exp["weather_lags"],
            model_config=model_config,
            years_split=years_split,
            evaluation_years=[2019],
            spike_config=spike_config,
            spike_eval_threshold=eval_spike_threshold,
            criterion=criterion,
            epochs=epochs
        )
        
        elapsed = time.time() - start_time
        
        # Add experiment config to result
        result["spike_trigger_threshold"] = exp["spike_trigger_threshold"]
        result["delta_lags"] = str(exp["delta_lags"])
        result["loss_type"] = exp["loss_type"]
        result["spike_weight"] = exp["spike_weight"]
        result["spike_lambda"] = exp["spike_lambda"]
        result["history_len"] = len(exp["history_offsets"])
        result["weather_lags"] = str(exp["weather_lags"])
        result["num_channels"] = str(model_config["num_channels"])
        result["kernel_size"] = model_config["kernel_size"]
        result["pooling"] = model_config["pooling"]
        result["dropout_encoder"] = model_config["dropout_encoder"]
        result["training_time_s"] = round(elapsed, 1)
        
        results.append(result)
        
        print(f"\n✓ {exp_name} completed in {elapsed:.1f}s")
        print(f"  best_val_loss: {result['best_val_loss']:.4f}")
        print(f"  best_epoch: {result['best_epoch']}")
        
    except Exception as e:
        elapsed = time.time() - start_time
        print(f"✗ FAILED: {exp_name} - {e}")
        results.append({
            "exp_name": exp_name, 
            "error": str(e),
            "training_time_s": round(elapsed, 1)
        })
    
    torch.cuda.empty_cache()

# ========================================================
# SAVE RESULTS
# ========================================================
os.makedirs(f"plots_training_dl/{broad_exp_name}/", exist_ok=True)
results_df = pd.DataFrame(results)
results_df.to_csv(f"plots_training_dl/{broad_exp_name}/batch_15_results.csv", index=False)

# Display summary
print("\n\n" + "="*80)
print("BATCH 15 EXPERIMENTS COMPLETE")
print("="*80)

display_cols = [
    "exp_name", "best_val_loss", "best_epoch", 
    "loss_type", "spike_weight", "spike_lambda", "history_len"
]
display_cols = [c for c in display_cols if c in results_df.columns]
print(results_df[display_cols].to_string())

# Show spike metrics if available
spike_cols = ["exp_name", "spike_recall", "spike_precision", "spike_f1", "spike_direction_accuracy"]
spike_cols = [c for c in spike_cols if c in results_df.columns]
if len(spike_cols) > 1:
    print("\n--- Spike Metrics ---")
    print(results_df[spike_cols].to_string())

print(f"\nResults saved to: plots_training_dl/{broad_exp_name}/batch_15_results.csv")


Running: EXP_spikeThr_0.04__lags_1-2-4-6__noAbs
  spike_trigger_threshold: 0.04
  delta_lags: [1, 2, 4, 6]
  loss_type: spike_weighted
  spike_weight: 3.0, spike_lambda: None
  history_offsets: 24h
  weather_lags: [0, -3, -6, -12, -24]
  model_config: channels=(128, 256, 256), k=5, pool=last
Loading data...
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h', 'rolling_vol_3h', 'is_high_vol']
Sequences created. Features: 42, Train samples: 1067911


Validating: 100%|██████████| 315/315 [00:04<00:00, 63.51it/s]


Epoch 1/10 - Train Loss: 0.2300 - Val Loss: 0.1508


Validating: 100%|██████████| 315/315 [00:05<00:00, 62.00it/s]


Epoch 2/10 - Train Loss: 0.1740 - Val Loss: 0.1439


...

Epoch 7/10 - Train Loss: 0.1379 - Val Loss: 0.1373


Validating: 100%|██████████| 315/315 [00:05<00:00, 62.03it/s]


Epoch 8/10 - Train Loss: 0.1338 - Val Loss: 0.1383


Epoch 9/10 - training:  16%|█▋        | 342/2085 [00:12<01:04, 27.04it/s]


KeyboardInterrupt: 

In [17]:
# Experiments on training parameters with base config

import time

broad_exp_name = "TCN-20det_24horizon-scheduler_optim_experiments"

# ========================================================
# DATA CONFIGURATION
# ========================================================
feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed",
    "is_weekend", "is_holiday", "is_school_holiday", "is_rush_hour", "is_snow", "is_fog"
]

nb_detectors = 20
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]
forecast_horizon = 24
epochs = 20

# ========================================================
# SPIKE FEATURE CONFIGURATION
# ========================================================
spike_config = SpikeFeatureConfig(
    enable_deltas=True,
    enable_abs_deltas=False,
    enable_rolling_stats=False,
    delta_lags=[1, 2, 4, 6],
    enable_volatility=True,
    volatility_window=3,
    volatility_binary_threshold=0.04
)

# ========================================================
# HISTORY & WEATHER LAGS
# ========================================================
h_offsets = list(range(24))
w_lags = [0, -3, -6, -12, -24]

# ========================================================
# LOSS CONFIGURATION
# ========================================================
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
cfg_loss = LossConfig(
    loss_type="spike_weighted",
    spike_weight=3.0,
    spike_threshold=spike_trigger_threshold
)

# ========================================================
# MODEL CONFIGURATION
# ========================================================
model_config = {
    "horizon": forecast_horizon,
    "num_detectors": nb_detectors,
    "emb_dim": 256,
    "num_channels": (128, 256, 256),
    "kernel_size": 5,
    "dropout_encoder": 0.1,
    "dropout_heads": 0.1,
    "use_se": False,
    "pooling": "last"
}

# ========================================================
# EXPERIMENT DEFINITIONS
# ========================================================
experiments = [
    {
        "name": "adamw_lr5e-4_no_scheduler",
        "optim_config": {"type": "adamW", "lr": 5e-4, "weight_decay": 1e-4},
        "schedule_config": None,
    },
    {
        "name": "adamw_lr5e-4_cosine",
        "optim_config": {"type": "adamW", "lr": 5e-4, "weight_decay": 1e-4},
        "schedule_config": {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6},
    },
    {
        "name": "adamw_lr5e-4_cosine_warm_restarts",
        "optim_config": {"type": "adamW", "lr": 5e-4, "weight_decay": 1e-4},
        "schedule_config": {"type": "CosineAnnealingWarmRestarts", "T_0": 5, "T_mult": 1, "eta_min": 1e-6},
    },
    {
        "name": "adamw_lr5e-4_onecycle",
        "optim_config": {"type": "adamW", "lr": 5e-4, "weight_decay": 1e-4},
        "schedule_config": {"type": "OneCycleLR", "max_lr": 1e-3, "epochs": epochs, "steps_per_epoch": 1},
    },
]

# ========================================================
# RUN EXPERIMENTS
# ========================================================
results = []

for exp in experiments:
    exp_name = exp["name"]
    print(f"\n{'='*70}")
    print(f"Running: {exp_name}")
    print(f"{'='*70}")
    print(f"  optim_config: {exp['optim_config']}")
    print(f"  schedule_config: {exp['schedule_config']}")
    
    start_time = time.time()
    criterion = create_loss(cfg_loss)
    
    try:
        result, model, losses = main(
            nb_detectors=nb_detectors,
            forecast_horizon=forecast_horizon,
            history_offsets=h_offsets,
            exp_name=exp_name,
            df_base=df_base,
            feature_cols_norm=feature_cols_norm_base,
            feature_cols=feature_cols_base,
            weather_lags=w_lags,
            model_config=model_config,
            years_split=years_split,
            evaluation_years=[2019],
            spike_config=spike_config,
            spike_eval_threshold=eval_spike_threshold,
            criterion=criterion,
            epochs=epochs,
            optim_config=exp["optim_config"],
            schedule_config=exp["schedule_config"]
        )
        
        elapsed = time.time() - start_time
        
        # Add experiment config to result
        result["optimizer"] = exp["optim_config"]["type"]
        result["lr"] = exp["optim_config"]["lr"]
        result["scheduler"] = exp["schedule_config"]["type"] if exp["schedule_config"] else "None"
        result["training_time_s"] = round(elapsed, 1)
        results.append(result)
        
        print(f"\n✓ {exp_name} completed in {elapsed:.1f}s")
        print(f"  best_val_loss: {result['best_val_loss']:.4f}")
        print(f"  best_epoch: {result['best_epoch']}")
        
    except Exception as e:
        elapsed = time.time() - start_time
        print(f"✗ FAILED: {exp_name} - {e}")
        import traceback
        traceback.print_exc()
        results.append({
            "exp_name": exp_name, 
            "error": str(e),
            "training_time_s": round(elapsed, 1)
        })
    
    torch.cuda.empty_cache()

# ========================================================
# SAVE RESULTS
# ========================================================
os.makedirs(f"plots_training_dl/{broad_exp_name}/", exist_ok=True)
results_df = pd.DataFrame(results)
results_df.to_csv(f"plots_training_dl/{broad_exp_name}/train_params_results.csv", index=False)

print("\n\n" + "="*80)
print("TRAINING PARAMS EXPERIMENTS COMPLETE")
print("="*80)

display_cols = ["exp_name", "optimizer", "lr", "scheduler", "best_val_loss", "best_epoch"]
display_cols = [c for c in display_cols if c in results_df.columns]
print(results_df[display_cols].to_string())

print(f"\nResults saved to: plots_training_dl/{broad_exp_name}/train_params_results.csv")


Running: adamw_lr5e-4_no_scheduler
  optim_config: {'type': 'adamW', 'lr': 0.0005, 'weight_decay': 0.0001}
  schedule_config: None
Loading data...
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h', 'rolling_vol_3h', 'is_high_vol']
Sequences created. Features: 42, Train samples: 1067911


Validating: 100%|██████████| 315/315 [00:05<00:00, 62.63it/s]


Epoch 1/20 - Train Loss: 0.1637 - Val Loss: 0.1241


Validating: 100%|██████████| 315/315 [00:05<00:00, 61.51it/s]


Epoch 2/20 - Train Loss: 0.1341 - Val Loss: 0.1160


...

Validating: 100%|██████████| 315/315 [00:05<00:00, 61.39it/s]


Epoch 20/20 - Train Loss: 0.0834 - Val Loss: 0.1166
<class 'pandas.core.frame.DataFrame'>
Index: 161980 entries, 10086243 to 13107649
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   hour_sin                161980 non-null  float64       
 1   hour_cos                161980 non-null  float64       
 2   dow_sin                 161980 non-null  float64       
 3   dow_cos                 161980 non-null  float64       
 4   month_sin               161980 non-null  float64       
 5   month_cos               161980 non-null  float64       
 6   lon                     161980 non-null  float64       
 7   lat                     161980 non-null  float64       
 8   year                    161980 non-null  float64       
 9   season                  161980 non-null  float64       
 10  temperature             161980 non-null  float64       
 11  precipitation           161980 non-

Epoch 1/20 - training:   1%|          | 20/2085 [00:00<01:26, 23.97it/s]

✗ FAILED: adamw_lr5e-4_onecycle - Tried to step 21 times. The specified number of total steps is 20


TRAINING PARAMS EXPERIMENTS COMPLETE
                            exp_name optimizer      lr                    scheduler  best_val_loss  best_epoch
0          adamw_lr5e-4_no_scheduler     adamW  0.0005                         None       0.113558         4.0
1                adamw_lr5e-4_cosine     adamW  0.0005            CosineAnnealingLR       0.112410         6.0
2  adamw_lr5e-4_cosine_warm_restarts     adamW  0.0005  CosineAnnealingWarmRestarts       0.113117         3.0
3              adamw_lr5e-4_onecycle       NaN     NaN                          NaN            NaN         NaN

Results saved to: plots_training_dl/TCN-20det_24horizon-scheduler_optim_experiments/train_params_results.csv



Traceback (most recent call last):
  File "/tmp/ipykernel_3687863/3162437689.py", line 114, in <module>
    result, model, losses = main(
                            ^^^^^
  File "/tmp/ipykernel_3687863/2036692409.py", line 69, in main
    model, train_losses, val_losses = run_dl_experiment(**params_experiment, exp_name=exp_name)
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3687863/504360682.py", line 64, in run_dl_experiment
    train_losses, val_losses, best_state = train_model(
                                           ^^^^^^^^^^^^
  File "/Data/CongestionAI/src/model_pipelines/dl_pipeline.py", line 86, in train_model
    scheduler.step()  # called every batch
    ^^^^^^^^^^^^^^^^
  File "/Data/CongestionAI/venv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py", line 238, in step
    values = self.get_lr()
             ^^^^^^^^^^^^^
  File "/Data/CongestionAI/venv/lib/python3.11/site-packages/torch/o

In [22]:
# Experiments on optimizers

import time

broad_exp_name = "TCN-20det_24horizon-optimizer_experiments"

# ========================================================
# DATA CONFIGURATION
# ========================================================
feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed",
    "is_weekend", "is_holiday", "is_school_holiday", "is_rush_hour", "is_snow", "is_fog"
]

nb_detectors = 20
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]
forecast_horizon = 24
epochs = 15

# ========================================================
# SPIKE FEATURE CONFIGURATION
# ========================================================
spike_config = SpikeFeatureConfig(
    enable_deltas=True,
    enable_abs_deltas=False,
    enable_rolling_stats=False,
    delta_lags=[1, 2, 4, 6],
    enable_volatility=True,
    volatility_window=3,
    volatility_binary_threshold=0.04
)

# ========================================================
# HISTORY & WEATHER LAGS
# ========================================================
h_offsets = list(range(24))
w_lags = [0, -3, -6, -12, -24]

# ========================================================
# LOSS CONFIGURATION
# ========================================================
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
cfg_loss = LossConfig(
    loss_type="spike_weighted",
    spike_weight=3.0,
    spike_threshold=spike_trigger_threshold
)

# ========================================================
# MODEL CONFIGURATION
# ========================================================
model_config = {
    "horizon": forecast_horizon,
    "num_detectors": nb_detectors,
    "emb_dim": 256,
    "num_channels": (128, 256, 256),
    "kernel_size": 5,
    "dropout_encoder": 0.1,
    "dropout_heads": 0.1,
    "use_se": False,
    "pooling": "last"
}

# ========================================================
# EXPERIMENT DEFINITIONS
# ========================================================
experiments = [
    # A. ADAMW VARIANTS
    {
        "name": "adamw_lr5e-4_betas_0.9_0.95",
        "optim_config": {"type": "adamW", "lr": 5e-4, "weight_decay": 1e-4, "betas": (0.9, 0.95)},
        "schedule_config": {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6},
    },
    {
        "name": "adamw_lr3e-4_betas_0.9_0.99",
        "optim_config": {"type": "adamW", "lr": 3e-4, "weight_decay": 1e-4, "betas": (0.9, 0.99)},
        "schedule_config": {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6},
    },
    
    # B. ADAM (non-decoupled weight decay)
    {
        "name": "adam_lr5e-4",
        "optim_config": {"type": "adam", "lr": 5e-4},
        "schedule_config": {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6},
    },
    {
        "name": "adam_lr3e-4",
        "optim_config": {"type": "adam", "lr": 3e-4},
        "schedule_config": {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6},
    },
    
    # C. RADAM
    {
        "name": "radam_lr5e-4",
        "optim_config": {"type": "radam", "lr": 5e-4},
        "schedule_config": {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6},
    },
    {
        "name": "radam_lr3e-4",
        "optim_config": {"type": "radam", "lr": 3e-4},
        "schedule_config": {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6},
    },
    
    # D. LION (very aggressive - use lower LR)
    {
        "name": "lion_lr1e-4",
        "optim_config": {"type": "lion", "lr": 1e-4, "weight_decay": 1e-4},
        "schedule_config": {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6},
    },
    {
        "name": "lion_lr5e-5",
        "optim_config": {"type": "lion", "lr": 5e-5, "weight_decay": 1e-4},
        "schedule_config": {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6},
    },
]

# ========================================================
# UPDATE MAIN TO SUPPORT NEW OPTIMIZERS
# ========================================================
def create_optimizer(model, optim_config):
    """Create optimizer based on config."""
    opt_type = optim_config["type"].lower()
    lr = optim_config.get("lr", 1e-4)
    weight_decay = optim_config.get("weight_decay", 0)
    betas = optim_config.get("betas", (0.9, 0.999))
    
    if opt_type == "adam":
        return torch.optim.Adam(model.parameters(), lr=lr, betas=betas)
    elif opt_type == "adamw":
        return torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay, betas=betas)
    elif opt_type == "radam":
        return torch.optim.RAdam(model.parameters(), lr=lr, betas=betas)
    elif opt_type == "lion":
        try:
            from lion_pytorch import Lion
            return Lion(model.parameters(), lr=lr, weight_decay=weight_decay)
        except ImportError:
            print("Lion not installed. Run: pip install lion-pytorch")
            raise
    else:
        raise ValueError(f"Unsupported optimizer type: {opt_type}")

# ========================================================
# RUN EXPERIMENTS
# ========================================================
results = []

for exp in experiments:
    exp_name = exp["name"]
    print(f"\n{'='*70}")
    print(f"Running: {exp_name}")
    print(f"{'='*70}")
    print(f"  optim_config: {exp['optim_config']}")
    print(f"  schedule_config: {exp['schedule_config']}")
    
    start_time = time.time()
    criterion = create_loss(cfg_loss)
    
    try:
        # Prepare data
        X_train_hist, Y_train, idx_train, det_train, \
        X_val_hist, Y_val, idx_val, det_val, \
        X_test_hist, Y_test, idx_test, det_test, \
        train, val, test, \
        std_scaler, mm_scaler = prepare_dl_data_with_spikes(
            h_offsets, forecast_horizon, nb_detectors, df_base,
            feature_cols_norm=feature_cols_norm_base,
            feature_cols_base=feature_cols_base,
            weather_lags=w_lags,
            years_split=years_split,
            spike_config=spike_config
        )
        
        # Create model
        model = MultiHeadTCNForecaster(**model_config, num_features=X_train_hist.shape[-1])
        
        # Create optimizer using new function
        optim = create_optimizer(model, exp["optim_config"])
        
        # Create scheduler
        batch_size = 512
        if exp["schedule_config"] is not None:
            sched_config = exp["schedule_config"].copy()
            if sched_config.get("type") == "OneCycleLR":
                steps_per_epoch = len(Y_train) // batch_size
                sched_config["steps_per_epoch"] = steps_per_epoch
            sched_class = getattr(torch.optim.lr_scheduler, sched_config["type"])
            sched_params = {k: v for k, v in sched_config.items() if k != "type"}
            scheduler = sched_class(optim, **sched_params)
        else:
            scheduler = None
        
        # Run experiment
        model, train_losses, val_losses = run_dl_experiment(
            model=model,
            optimizer=optim,
            criterion=criterion,
            X_train_hist=X_train_hist,
            Y_train=Y_train,
            train_det_idx=det_train,
            X_val_hist=X_val_hist,
            Y_val=Y_val,
            val_det_idx=det_val,
            X_test_hist=X_test_hist,
            Y_test=Y_test,
            test_det_idx=det_test,
            device="cuda",
            batch_size=batch_size,
            epochs=epochs,
            grad_clip=None,
            scheduler=scheduler,
            exp_name=exp_name
        )
        
        # Evaluate
        eval_df = prepare_eval_df(val, idx_val, predict(model, X_val_hist, det_val), forecast_horizon)
        eval_df["congestion_index"] = val.loc[idx_val, "congestion_index"].values
        metrics = evaluate_and_plot_block(
            eval_df, horizon=forecast_horizon, years=[2019], plot_years=[2019],
            filename=exp_name, dir=f"plots_training_dl/{broad_exp_name}/",
            max_blocks=15, eval_spikes=True, spike_threshold=eval_spike_threshold
        )
        
        elapsed = time.time() - start_time
        
        # Build result
        result = {
            "exp_name": exp_name,
            "num_features": X_train_hist.shape[-1],
            "train_samples": len(Y_train),
            "val_samples": len(Y_val),
            "final_train_loss": train_losses[-1],
            "final_val_loss": val_losses[-1],
            "best_val_loss": min(val_losses),
            "best_epoch": val_losses.index(min(val_losses)) + 1,
            "optimizer": exp["optim_config"]["type"],
            "lr": exp["optim_config"]["lr"],
            "betas": str(exp["optim_config"].get("betas", "(0.9, 0.999)")),
            "scheduler": exp["schedule_config"]["type"] if exp["schedule_config"] else "None",
            "training_time_s": round(elapsed, 1)
        }
        
        # Add metrics
        if metrics is not None:
            for k, v in metrics.items():
                if isinstance(v, (int, float)):
                    result[k] = v
        
        results.append(result)
        
        print(f"\n✓ {exp_name} completed in {elapsed:.1f}s")
        print(f"  best_val_loss: {result['best_val_loss']:.4f}")
        print(f"  best_epoch: {result['best_epoch']}")
        
    except Exception as e:
        elapsed = time.time() - start_time
        print(f"✗ FAILED: {exp_name} - {e}")
        import traceback
        traceback.print_exc()
        results.append({
            "exp_name": exp_name, 
            "error": str(e),
            "training_time_s": round(elapsed, 1)
        })
    
    torch.cuda.empty_cache()

# ========================================================
# SAVE RESULTS
# ========================================================
os.makedirs(f"plots_training_dl/{broad_exp_name}/", exist_ok=True)
results_df = pd.DataFrame(results)
results_df.to_csv(f"plots_training_dl/{broad_exp_name}/optimizer_results.csv", index=False)

print("\n\n" + "="*80)
print("OPTIMIZER EXPERIMENTS COMPLETE")
print("="*80)

display_cols = ["exp_name", "optimizer", "lr", "betas", "best_val_loss", "best_epoch"]
display_cols = [c for c in display_cols if c in results_df.columns]
print(results_df[display_cols].to_string())

# Show spike metrics
spike_cols = ["exp_name", "spike_recall", "spike_precision", "spike_f1", "spike_direction_accuracy"]
spike_cols = [c for c in spike_cols if c in results_df.columns]
if len(spike_cols) > 1:
    print("\n--- Spike Metrics ---")
    print(results_df[spike_cols].to_string())

print(f"\nResults saved to: plots_training_dl/{broad_exp_name}/optimizer_results.csv")


Running: adamw_lr5e-4_betas_0.9_0.95
  optim_config: {'type': 'adamW', 'lr': 0.0005, 'weight_decay': 0.0001, 'betas': (0.9, 0.95)}
  schedule_config: {'type': 'CosineAnnealingLR', 'T_max': 15, 'eta_min': 1e-06}
Loading data...
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h', 'rolling_vol_3h', 'is_high_vol']
Sequences created. Features: 42, Train samples: 1067911


Validating: 100%|██████████| 315/315 [00:04<00:00, 63.18it/s]


Epoch 1/15 - Train Loss: 0.1689 - Val Loss: 0.1228


Validating: 100%|██████████| 315/315 [00:05<00:00, 62.95it/s]


Epoch 2/15 - Train Loss: 0.1374 - Val Loss: 0.1178


...

Epoch 13/15 - Train Loss: 0.0933 - Val Loss: 0.1148


Validating: 100%|██████████| 315/315 [00:04<00:00, 63.13it/s]


Epoch 14/15 - Train Loss: 0.0917 - Val Loss: 0.1147


Validating: 100%|██████████| 315/315 [00:04<00:00, 63.11it/s]


Epoch 15/15 - Train Loss: 0.0902 - Val Loss: 0.1146
<class 'pandas.core.frame.DataFrame'>
Index: 161980 entries, 10086243 to 13107649
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   hour_sin                161980 non-null  float64       
 1   hour_cos                161980 non-null  float64       
 2   dow_sin                 161980 non-null  float64       
 3   dow_cos                 161980 non-null  float64       
 4   month_sin               161980 non-null  float64       
 5   month_cos               161980 non-null  float64       
 6   lon                     161980 non-null  float64       
 7   lat                     161980 non-null  float64       
 8   year                    161980 non-null  float64       
 9   season                  161980 non-null  float64       
 10  temperature             161980 non-null  float64       
 11  precipitation           161980 non-

In [7]:
# Scale experiment: 100 detectors with best regularization config

import time

broad_exp_name = "TCN-100det_24horizon-best_regularization"

# ========================================================
# DATA CONFIGURATION
# ========================================================
feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed",
    "is_weekend", "is_holiday", "is_school_holiday", "is_rush_hour", "is_snow", "is_fog"
]

nb_detectors = 100
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]
forecast_horizon = 24
epochs = 20

# ========================================================
# SPIKE FEATURE CONFIGURATION
# ========================================================
spike_config = SpikeFeatureConfig(
    enable_deltas=True,
    enable_abs_deltas=False,
    enable_rolling_stats=False,
    delta_lags=[1, 2, 4, 6],
    enable_volatility=True,
    volatility_window=3,
    volatility_binary_threshold=0.04
)

# ========================================================
# HISTORY & WEATHER LAGS
# ========================================================
h_offsets = list(range(24))
w_lags = [0, -3, -6, -12, -24]

# ========================================================
# LOSS CONFIGURATION
# ========================================================
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
cfg_loss = LossConfig(
    loss_type="spike_weighted",
    spike_weight=3.0,
    spike_threshold=spike_trigger_threshold
)

# ========================================================
# MODEL CONFIGURATION (best regularization)
# ========================================================
model_config = {
    "horizon": forecast_horizon,
    "num_detectors": nb_detectors,
    "emb_dim": 256,
    "num_channels": (128, 256, 256),
    "kernel_size": 5,
    "dropout_encoder": 0.25,
    "dropout_heads": 0.25,
    "use_se": False,
    "pooling": "last"
}

# ========================================================
# OPTIMIZER & SCHEDULER CONFIG
# ========================================================
optim_config = {"type": "adamW", "lr": 3e-4, "weight_decay": 1e-3, "betas": (0.9, 0.99)}
schedule_config = {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6}
grad_clip = 0.5

# ========================================================
# RUN EXPERIMENT
# ========================================================
exp_name = "100det_aggressive_reg"
print(f"\n{'='*70}")
print(f"Running: {exp_name}")
print(f"{'='*70}")
print(f"  nb_detectors: {nb_detectors}")
print(f"  dropout_encoder: {model_config['dropout_encoder']}")
print(f"  dropout_heads: {model_config['dropout_heads']}")
print(f"  weight_decay: {optim_config['weight_decay']}")
print(f"  grad_clip: {grad_clip}")

start_time = time.time()
criterion = create_loss(cfg_loss)

# Prepare data
X_train_hist, Y_train, idx_train, det_train, \
X_val_hist, Y_val, idx_val, det_val, \
X_test_hist, Y_test, idx_test, det_test, \
train, val, test, \
std_scaler, mm_scaler = prepare_dl_data_with_spikes(
    h_offsets, forecast_horizon, nb_detectors, df_base,
    feature_cols_norm=feature_cols_norm_base,
    feature_cols_base=feature_cols_base,
    weather_lags=w_lags,
    years_split=years_split,
    spike_config=spike_config
)

# Create model
model = MultiHeadTCNForecaster(**model_config, num_features=X_train_hist.shape[-1])

# Create optimizer
optim = torch.optim.AdamW(
    model.parameters(), 
    lr=optim_config["lr"], 
    weight_decay=optim_config["weight_decay"], 
    betas=optim_config["betas"]
)

# Create scheduler
batch_size = 512
sched_class = getattr(torch.optim.lr_scheduler, schedule_config["type"])
sched_params = {k: v for k, v in schedule_config.items() if k != "type"}
scheduler = sched_class(optim, **sched_params)

# Run experiment
model, train_losses, val_losses = run_dl_experiment(
    model=model,
    optimizer=optim,
    criterion=criterion,
    X_train_hist=X_train_hist,
    Y_train=Y_train,
    train_det_idx=det_train,
    X_val_hist=X_val_hist,
    Y_val=Y_val,
    val_det_idx=det_val,
    X_test_hist=X_test_hist,
    Y_test=Y_test,
    test_det_idx=det_test,
    device="cuda",
    batch_size=batch_size,
    epochs=epochs,
    grad_clip=grad_clip,
    scheduler=scheduler,
    exp_name=exp_name
)

# Evaluate
eval_df = prepare_eval_df(val, idx_val, predict(model, X_val_hist, det_val), forecast_horizon)
eval_df["congestion_index"] = val.loc[idx_val, "congestion_index"].values
metrics = evaluate_and_plot_block(
    eval_df, horizon=forecast_horizon, years=[2019], plot_years=[2019],
    filename=exp_name, dir=f"plots_training_dl/{broad_exp_name}/",
    max_blocks=15, eval_spikes=True, spike_threshold=eval_spike_threshold
)

elapsed = time.time() - start_time

# Print results
print(f"\n{'='*70}")
print(f"RESULT: {exp_name}")
print(f"{'='*70}")
print(f"Training time: {elapsed:.1f}s")
print(f"Train samples: {len(Y_train)}")
print(f"Val samples: {len(Y_val)}")
print(f"Best val loss: {min(val_losses):.4f}")
print(f"Best epoch: {val_losses.index(min(val_losses)) + 1}")
if metrics:
    print(f"Spike recall: {metrics.get('spike_recall', 'N/A')}")
    print(f"Spike precision: {metrics.get('spike_precision', 'N/A')}")
    print(f"Spike F1: {metrics.get('spike_f1', 'N/A')}")


Running: 100det_aggressive_reg
  nb_detectors: 100
  dropout_encoder: 0.25
  dropout_heads: 0.25
  weight_decay: 0.001
  grad_clip: 0.5
Loading data...
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h', 'rolling_vol_3h', 'is_high_vol']
Sequences created. Features: 42, Train samples: 4872519


Validating: 100%|██████████| 1513/1513 [00:24<00:00, 61.30it/s]


Epoch 1/20 - Train Loss: 0.3084 - Val Loss: 0.2457


Validating: 100%|██████████| 1513/1513 [00:24<00:00, 60.67it/s]


Epoch 2/20 - Train Loss: 0.2704 - Val Loss: 0.2347


...

Epoch 18/20 - Train Loss: 0.2244 - Val Loss: 0.2292


Validating: 100%|██████████| 1513/1513 [00:24<00:00, 61.40it/s]


Epoch 19/20 - Train Loss: 0.2235 - Val Loss: 0.2318


Validating: 100%|██████████| 1513/1513 [00:25<00:00, 60.22it/s]


Epoch 20/20 - Train Loss: 0.2228 - Val Loss: 0.2316
<class 'pandas.core.frame.DataFrame'>
Index: 779017 entries, 10086243 to 13108075
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   hour_sin                779017 non-null  float64       
 1   hour_cos                779017 non-null  float64       
 2   dow_sin                 779017 non-null  float64       
 3   dow_cos                 779017 non-null  float64       
 4   month_sin               779017 non-null  float64       
 5   month_cos               779017 non-null  float64       
 6   lon                     779017 non-null  float64       
 7   lat                     779017 non-null  float64       
 8   year                    779017 non-null  float64       
 9   season                  779017 non-null  float64       
 10  temperature             779017 non-null  float64       
 11  precipitation           779017 non-

In [7]:
# Scale experiment: 100 detectors with OPTIMIZATIONS

import time
import torch
from torch.amp import GradScaler

torch.set_float32_matmul_precision('high')

broad_exp_name = "TCN-100det_24horizon-best_regularization_OPTIMIZED"

# ========================================================
# DATA CONFIGURATION
# ========================================================
feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed",
    "is_weekend", "is_holiday", "is_school_holiday", "is_rush_hour", "is_snow", "is_fog"
]

nb_detectors = 100
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]
forecast_horizon = 24
epochs = 20

# ========================================================
# SPIKE FEATURE CONFIGURATION
# ========================================================
spike_config = SpikeFeatureConfig(
    enable_deltas=True,
    enable_abs_deltas=False,
    enable_rolling_stats=False,
    delta_lags=[1, 2, 4, 6],
    enable_volatility=True,
    volatility_window=3,
    volatility_binary_threshold=0.04
)

# ========================================================
# HISTORY & WEATHER LAGS
# ========================================================
h_offsets = list(range(24))
w_lags = [0, -3, -6, -12, -24]

# ========================================================
# LOSS CONFIGURATION
# ========================================================
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
cfg_loss = LossConfig(
    loss_type="spike_weighted",
    spike_weight=3.0,
    spike_threshold=spike_trigger_threshold
)

# ========================================================
# MODEL CONFIGURATION (best regularization)
# ========================================================
model_config = {
    "horizon": forecast_horizon,
    "num_detectors": nb_detectors,
    "emb_dim": 256,
    "num_channels": (128, 256, 256),
    "kernel_size": 5,
    "dropout_encoder": 0.25,
    "dropout_heads": 0.25,
    "use_se": False,
    "pooling": "last"
}

# ========================================================
# OPTIMIZER & SCHEDULER CONFIG
# ========================================================
optim_config = {"type": "adamW", "lr": 3e-4, "weight_decay": 1e-3, "betas": (0.9, 0.99)}
schedule_config = {"type": "CosineAnnealingLR", "T_max": epochs, "eta_min": 1e-6}
grad_clip = 0.5

# ========================================================
# OPTIMIZATION SETTINGS
# ========================================================
batch_size = 2048  # Increased from 512 (AMP allows larger batches)
num_workers = 4    # Parallel data loading
use_amp = True     # Mixed precision training
use_compile = True # torch.compile (PyTorch 2.0+)

# ========================================================
# OPTIMIZED run_dl_experiment FUNCTION
# ========================================================
def run_dl_experiment_optimized(
    model,
    optimizer,
    criterion,
    X_train_hist,
    Y_train,
    train_det_idx,
    X_val_hist,
    Y_val,
    val_det_idx,
    X_test_hist,
    Y_test,
    test_det_idx,
    device="cuda",
    batch_size=512,
    epochs=10,
    grad_clip=1.0,
    scheduler=None,
    scaler=None,
    exp_name="",
    patience=None,
    num_workers=4,
):
    """Optimized training pipeline with multi-worker dataloaders."""

    # -------------------------
    # OPTIMIZED DATALOADERS
    # -------------------------
    train_loader = DataLoader(
        NHitsDataset(X_train_hist, Y_train, train_det_idx),
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        pin_memory=True,
        num_workers=num_workers,
        persistent_workers=True if num_workers > 0 else False,
        prefetch_factor=2 if num_workers > 0 else None,
    )

    val_loader = DataLoader(
        NHitsDataset(X_val_hist, Y_val, val_det_idx),
        batch_size=batch_size * 2,  # Larger batch for eval (no gradients)
        shuffle=False,
        pin_memory=True,
        num_workers=num_workers,
        persistent_workers=True if num_workers > 0 else False,
        prefetch_factor=2 if num_workers > 0 else None,
    )

    if X_test_hist is None or Y_test is None or test_det_idx is None:
        test_loader = None
    else:
        test_loader = DataLoader(
            NHitsDataset(X_test_hist, Y_test, test_det_idx),
            batch_size=batch_size * 2,
            shuffle=False,
            pin_memory=True,
            num_workers=num_workers,
        )

    model.to(device)

    # -------------------------
    # TRAINING
    # -------------------------
    train_losses, val_losses, best_state = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        scaler=scaler,
        device=device,
        num_epochs=epochs,
        grad_clip=grad_clip,
        patience=patience,
    )

    # Save losses to file
    os.makedirs(f"plots_training_dl/{broad_exp_name}/", exist_ok=True)
    with open(f"plots_training_dl/{broad_exp_name}/losses_{exp_name}.txt", "w") as f:
        f.write("epoch,train_loss,val_loss\n")
        for i, (t_loss, v_loss) in enumerate(zip(train_losses, val_losses)):
            f.write(f"{i+1},{t_loss:.6f},{v_loss:.6f}\n")

    plot_training_curves(train_losses, val_losses, filename=f"training_curve{exp_name}.png", dir=f"plots_training_dl/{broad_exp_name}/")
    
    if test_loader is not None:
        _, test_loss = evaluate(model, test_loader, criterion, device, scaler=scaler)
        print(f"Test Loss ({exp_name}): {test_loss:.4f}")

    return model, train_losses, val_losses

# ========================================================
# RUN EXPERIMENT
# ========================================================
exp_name = "100det_aggressive_reg_OPTIMIZED"
print(f"\n{'='*70}")
print(f"Running: {exp_name}")
print(f"{'='*70}")
print(f"  nb_detectors: {nb_detectors}")
print(f"  dropout_encoder: {model_config['dropout_encoder']}")
print(f"  dropout_heads: {model_config['dropout_heads']}")
print(f"  weight_decay: {optim_config['weight_decay']}")
print(f"  grad_clip: {grad_clip}")
print(f"  batch_size: {batch_size}")
print(f"  num_workers: {num_workers}")
print(f"  AMP enabled: {use_amp}")
print(f"  torch.compile: {use_compile}")

start_time = time.time()
criterion = create_loss(cfg_loss)

# Prepare data
print("\n[1/4] Preparing data...")
data_start = time.time()
X_train_hist, Y_train, idx_train, det_train, \
X_val_hist, Y_val, idx_val, det_val, \
X_test_hist, Y_test, idx_test, det_test, \
train, val, test, \
std_scaler, mm_scaler = prepare_dl_data_with_spikes(
    h_offsets, forecast_horizon, nb_detectors, df_base,
    feature_cols_norm=feature_cols_norm_base,
    feature_cols_base=feature_cols_base,
    weather_lags=w_lags,
    years_split=years_split,
    spike_config=spike_config
)
print(f"    Data prep time: {time.time() - data_start:.1f}s")

# Create model
print("\n[2/4] Creating model...")
model = MultiHeadTCNForecaster(**model_config, num_features=X_train_hist.shape[-1])

# Apply torch.compile for faster execution (PyTorch 2.0+)
if use_compile and hasattr(torch, 'compile'):
    print("    Applying torch.compile()...")
    model = torch.compile(model, mode="reduce-overhead")

# Create optimizer
optim = torch.optim.AdamW(
    model.parameters(), 
    lr=optim_config["lr"], 
    weight_decay=optim_config["weight_decay"], 
    betas=optim_config["betas"]
)

# Create scheduler
sched_class = getattr(torch.optim.lr_scheduler, schedule_config["type"])
sched_params = {k: v for k, v in schedule_config.items() if k != "type"}
scheduler = sched_class(optim, **sched_params)

# Create AMP scaler
scaler = GradScaler('cuda') if use_amp else None

# Run experiment
print("\n[3/4] Training...")
train_start = time.time()
model, train_losses, val_losses = run_dl_experiment_optimized(
    model=model,
    optimizer=optim,
    criterion=criterion,
    X_train_hist=X_train_hist,
    Y_train=Y_train,
    train_det_idx=det_train,
    X_val_hist=X_val_hist,
    Y_val=Y_val,
    val_det_idx=det_val,
    X_test_hist=X_test_hist,
    Y_test=Y_test,
    test_det_idx=det_test,
    device="cuda",
    batch_size=batch_size,
    epochs=epochs,
    grad_clip=grad_clip,
    scheduler=scheduler,
    scaler=scaler,
    exp_name=exp_name,
    num_workers=num_workers,
)
print(f"    Training time: {time.time() - train_start:.1f}s")

# Evaluate
print("\n[4/4] Evaluating...")
eval_df = prepare_eval_df(val, idx_val, predict(model, X_val_hist, det_val), forecast_horizon)
eval_df["congestion_index"] = val.loc[idx_val, "congestion_index"].values
metrics = evaluate_and_plot_block(
    eval_df, horizon=forecast_horizon, years=[2019], plot_years=[2019],
    filename=exp_name, dir=f"plots_training_dl/{broad_exp_name}/",
    max_blocks=15, eval_spikes=True, spike_threshold=eval_spike_threshold
)

elapsed = time.time() - start_time

# Print results
print(f"\n{'='*70}")
print(f"RESULT: {exp_name}")
print(f"{'='*70}")
print(f"Total time: {elapsed:.1f}s")
print(f"Train samples: {len(Y_train)}")
print(f"Val samples: {len(Y_val)}")
print(f"Best val loss: {min(val_losses):.4f}")
print(f"Best epoch: {val_losses.index(min(val_losses)) + 1}")
if metrics:
    print(f"Spike recall: {metrics.get('spike_recall', 'N/A')}")
    print(f"Spike precision: {metrics.get('spike_precision', 'N/A')}")
    print(f"Spike F1: {metrics.get('spike_f1', 'N/A')}")

print(f"\n--- OPTIMIZATION SUMMARY ---")
print(f"  AMP: {'enabled' if use_amp else 'disabled'}")
print(f"  torch.compile: {'enabled' if use_compile else 'disabled'}")
print(f"  DataLoader workers: {num_workers}")
print(f"  Batch size: {batch_size}")


Running: 100det_aggressive_reg_OPTIMIZED
  nb_detectors: 100
  dropout_encoder: 0.25
  dropout_heads: 0.25
  weight_decay: 0.001
  grad_clip: 0.5
  batch_size: 2048
  num_workers: 4
  AMP enabled: True
  torch.compile: True

[1/4] Preparing data...
Loading data...
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h', 'rolling_vol_3h', 'is_high_vol']
Sequences created. Features: 42, Train samples: 4872519
    Data prep time: 59.6s

[2/4] Creating model...
    Applying torch.compile()...

[3/4] Training...


Validating: 100%|██████████| 190/190 [00:14<00:00, 13.39it/s]


Epoch 1/20 - Train Loss: 0.3318 - Val Loss: 0.2521


Epoch 2/20 - training:  19%|█▉        | 451/2379 [00:24<01:44, 18.40it/s]


KeyboardInterrupt: 